# Apéndice B. Mecanismos de E/S en sistemas embebidos

Para implementar un algoritmo de procesamiento de señales en tiempo real, es necesario conectar dos dispositivos al procesador, el conversor analógico-digital (ADC) y el conversor digital-analógico (DAC). Tanto el ADC como el DAC deben sincronizarse para que cada un periodo de muestreo $T_s = 1/f_s$ se lea una nueva muestra del ADC, se procese y se genere una nueva muestra de salida al DAC. 

![Sistema Básico de Procesamiento Digital de Señales](../img/SistemaDSP.png)

Para conectar con el hardware ADC, DAC o cualquier dispositivo, todos los sistemas basados en procesador hacen uso de alguno de los siguientes tres mecanismos de Entrada/Salida (E/S):

1. **E/S Programada o _polling_**. El mecanismo de _polling_ es una técnica para manipulación de dispositivos de entrada/salida en la que una rutina de software continuamente verifica la presencia de un dato en el dispositivo de entrada, de existir, lee el dispositivo y procede al procesamiento del dato leído.

2.  **E/S por interrupción**. El mecanismo de interrupción es una técnica de acceso a dispositivos de entrada/salida superior al _polling_, en el que el hardware envía una solicitud al procesador para ejecutar una rutina de servicio de interrupción (ISR: _Interrupt Service Routine_). Esta es una rutina de software encargada de leer el dispositivo, y opcionalmente realizar el procesamiento. De esta forma, una interrupción puede verse como una rutina de software invocada por el hardware. La ventaja con respecto al _polling_ es que el procesador no tiene que estar revisando continuamente si hay un dato nuevo, en su lugar, pudiendo encargarse de otras de menor prioridad.

3. **E/S por acceso directo a memoria o DMA (_Direct Memory Access_)**. En este mecanismo, la adquisición de datos la realiza completamente el hardware sin intervención de rutinas de software, logrando así tasas de transferencia superiores a las técnicas de _polling_ e interrupción. Para lograrlo, el software debe programar el dispositivo y controlador de DMA para realizar una adquisición de N muestras de datos. Cuando los buses externos del procesador están libres, el controlador de DMA realiza la transferencia de datos desde el dispositivo a la memoria, y una vez se han transferido los N datos, el hardware invoca una rutina de servicio de interrupción, que se debe encargar, por software, del procesamiento de los datos. 

Los dos primeros mecanismos (_polling_ e interrupción) están disponibles en todos los microcontroladores, sin embargo, el DMA, por la complejidad del hardware requerido solamente está disponible en ciertos microcontroladores de alta gama (como el ESP32) o en dispositivos más complejos como los smartphone o computadores.

Debemos recordar que la forma natural de representar los algoritmos que corren dentro del DSP es a través de ecuaciones en diferencias (ED) y la ED se puede implementar de dos formas: **implementación orientada a muestras** e **implementación orientada a bloque**, como se mostró en el Capítulo 3. Aunque existen tres mecanismos de E/S disponibles para la implementación del sistema mostrado en la Figura de arriba, los mecanismos de _polling_ e interrupción resultan más conveniente para la implementación orientada a muestras, mientras que el DMA es más efectivo con una implementación orientada a bloque.

| Forma de implementación ED | E/S recomendado |
|------|------|
| Orientada a muestras | E/S _polling_ |
| Orientada a muestras | E/S por interrupción |
| Orientada a bloque | E/S por DMA |

Aunque la implementación orientada a bloque se podría usar con _polling_ e interrupción, no es muy conveniente, ya que esto implicaría usar _polling_ o interrupción para leer un bloque de N datos, y procesarlo una vez esté lleno, lo cual implicaría un tiempo de cómputo mayor que si hiciera muestra a muestra.

## B.1. Implementación Orientada a Muestras con E/S _Polling_

Supongamos que la ED se implementa usando el esquema orientado a muestras. En este esquema, cada vez que se lee una muestra del ADC, se procesa y el resultado se escribe inmediatamente al DAC. La estructura básica de un programa por E/S Polling para implementar un filtro digital luce como se muestra a continuación. 

```C
void loop() {
    float x = LeerADC();
    float y = filtrar(x);
    EscribirDAC(y);
    EsperarSiguienteTs();
}
```

En este fragmento de código, la función ``filtrar`` implementa la ED del filtro digital, para lo cual recibe una muestra y entrega una muestra. En los Capítulos 3 y 4 se muestra implementar una ED bajo el esquema de implementación orientada a muestras.

Para una aplicación en tiempo real se requiere que el tiempo de cómputo sea inferior al tiempo máximo permitido. En una implementación orientada a muestras, dicho tiempo máximo es igual al tiempo que tarda el algoritmo de pasar de una instrucción ``LeerADC`` a la siguiente ``LeerADC``, es decir, igual al periodo de muestreo $T_{s}=\frac{1}{f_{s}}$, por lo tanto, el tiempo de cómputo de la función ``filtrar`` no puede exceder la condición

$$T_{computo}<T_{s}$$

Es importante resaltar que en este algoritmo, se requiere garantizar que el ADC entregue datos a una tasa dada por el periodo de muestreo $T_s$. Programadores novatos podrían implementar el anterior código reemplazando la línea ``EsperarSiguienteTs()`` con un simple ``delay(Ts)``. Esto no funciona ya que la rutina ``filtrar`` toma un tiempo cómputo $T_{computo}$, por lo cual si se fuera a emplear una instrucción ``delay`` esta tendría que considerar que el tiempo de espera debe ser $$T_{delay} = T_s - T_{computo}$$ 

El siguiente programa de referencia en Wokwi muestra como el código anterior tiene que ser adaptado a una plataforma Arduino ESP32 (https://wokwi.com/projects/406323503999269889). Para que este algoritmo funcione es necesario hacer previamente un _profiling_ para medir el tiempo de cómputo de la función ``filter``.

```C
void loop() {
  //LeerADC: ADC tiene una resolución de 12-bit
  int x = analogRead(ADC_IN);
  int y = filter(x);
  //EscribirDAC: DAC tiene 8-bit de resolución, así que toca escalar la salida
  dacWrite(DAC1, y >> 4);
  delayMicroseconds(Ts-Tcomp);
}
```

Esta solución solamente funcionaría si dentro del ``loop`` no se hacen otras instrucciones como por ejemplo visualizar. Una forma más conveniente de garantizar un periodo de muestreo adecuado es medir el tiempo transcurrido entre dos ``LeerADC``. En Arduino, este se puede hacer fácilmente usando la función ``micros`` como se muestra a continuación.

```C
unsigned long previous_time; //Tiempo anterior
unsigned long Ts_us = 1000;  //Tiempo de muestreo en microsegundos
void setup() {
    previous_time = micros();
}
void loop() {
    unsigned long ctime = micros();
    while( previous_time-ctime < Ts_us ) {
        ctime = micros();
    }
    previous_time = ctime;
    int x = analogRead(ADC_IN);
    int y = filter(x);
    dacWrite(DAC1, y >> 2);
}
```

Nótese que en este código, la instrucción ``delay`` fue reemplazada por un ciclo ``while`` que verifica el tiempo transcurrido y espera lo que resta del tiempo para iniciar una nueva conversión del ADC, de allí que la ubicación del código haya tenido que ser reorganizada.

## B.2. Implementación Orientada a Muestras con E/S por interrupción

Como se ha indicado anteriormente, E/S por interrupción ofrece mejores prestaciones que E/S por _polling_ debido a un mejor control del periodo de muestreo y que se libera al procesador de estar revisando cuando hay una nueva muestra disponible. 

Para implementar este mecanismo basta con mover el código base que lee el ADC, procesa la señal y escribe al DAC a la rutina de servicio de interrupción (ISR). Adicionalmente, es necesario configurar la ISR para que se dispare cada periodo de muestreo. Esto último se consigue habitualmente con un timer como se muestra en el siguiente ejemplo.

```C
hw_timer_t* timer = NULL; //Hardware timer

void ARDUINO_ISR_ATTR isrTimer() {
    int x = analogRead(ADC_IN);
    int y = filter(x);
    dacWrite(DAC1, y>>4);
}

void setup() {
  //Configura el timer
  timer = timerBegin(1000000); // Fija la frecuencia de entrada al timer en 1MHz
  timerAttachInterrupt(timer, &isrTimer); // Conecta la ISR al timer
  //Fija el Ts. El tercer y cuarto parámetro le dicen que se repita indefinidamente
  timerAlarm(timer, Ts_us, true, 0); 
}

void loop() {
}
```

Note que en este ejemplo, el loop no hace nada, y se puede incluir código en este sitio para realizar otras funciones. El código de refernecia en Wokwi que muestra este enfoque se accede a través de https://wokwi.com/projects/406690352968126465. 

## B.3 Implementación Orientada a Bloque por E/S por DMA

De los mecanismos de E/S, el DMA ofrece las mejores características para la adquisición de bloques de datos en tiempo real, dado a que esta técnica libera al procesador de la tarea de leer/escribir datos desde el dispositivo hacia la memoria y viceversa. De esta forma, mientras el hardware de DMA transfiere los datos, el procesador se dedica en paralelo a procesar el bloque de información capturado anteriormente. Cuando se adquieren datos por DMA, al final de la transferencia se genera una interrupción en la que se debe procesar el bloque recién adquirido, de allí que más aconsejable es una implementación orientada a bloques para calcular el bloque de salida.

En el Capítulo 3 se mostró como implementar un filtro FIR (sumatoria de convolución) empleando procesamiento orientado a bloques y se mostró un programa ejemplo basado en DMA el cual se presenta a continuación.

```C
//Arreglos de entrada y salida usados en los cálculos
short x_blk[NBLOQUE + Nh-1];
short y_blk[NBLOQUE];

//Rutina que se llama cada vez que finaliza la captura de un bloque
void dma_callbackfnc(short* x_new) {
        int n, k;
        float y;

        //Copia el nuevo bloque de datos en x_blk a partir de la posición Nh-1 
        //con el fin de reservar las primeras posiciones para los elementos del bloque anterior
        //En ciertas plataformas, este proceso de copia se puede eliminar indicándole al 
        //hardware de DMA que inicie la captura a partir de la posición Nh-1 del bloque x_blk.
        for(n=0; n<NBLOQUE; n++) {
                x_blk[n+Nh-1] = x_new[n];         
        }

        //Le indica al hardware que inicie una nueva captura por DMA
        captureBlock(dma_callbackfnc);

        //Calcula las Nh convoluciones. El ajuste del índice Nh-1 se usa para evitar el acceso 
        //a índices negativos del buffer de entrada que corresponden a las muestras del bloque pasado 
        for(n=0; n<NBLOQUE; n++)  {
                y = 0;
                for(k=0; k<Nh; k++)  {
                        y += h[k] * x_blk[n-k+Nh-1];
                }
                y_blk[n] = y;
        }

        //Envía el bloque procesado al DAC 
        playBlock(y_blk);

        //Copia los últimos elementos del bloque de entrada al principio para preparar
        //el bloque para la siguiente iteración
        for(k=0; k<Nh-1; k++)  {
                x_blk[k] = x_blk[k+NBLOQUE];
        }
}
```

Cuando se trabaja con DMA en aplicaciones de tiempo real, se hace necesario garantizar que los bloques capturados correspondan realmente a tramos continuos de la señal adquirida y el procesamiento de los bloques de información se haga a una velocidad similar o más rápida que el tiempo de captura del bloque; la única forma de hacerlo es por medio de la técnica de **doble buffer**. En doble buffer, el programa principal hace la solicitud de transferencia de únicamente un bloque, y cuando se genera la interrupción por fin de transferencia, dicha rutina ISR se encarga de hacer la solicitud de captura del siguiente bloque y procesar el bloque capturado, garantizando de esta forma una continuidad en la captura y procesamiento de la señal. En el código de arriba, la función ``dma_callbackfnc`` es justamente la ISR encargada de la captura, procesamiento y escritura en el DAC.

En el código de la ISR presentado arriba, el doble buffer se implementa copiando el bloque recién adquirido ``x_new`` en ``x_blk`` para posteriormente hacer el procesamiento sobre ``x_blk``. Adicionalmente, nótese que se invoca la función ``captureBlock`` antes de iniciar el filtrado, lo cual permite que en paralelo mientras se calcula el nuevo bloque se capture el siguiente por medio de DMA.

En lo que respecta a tiempos de ejecución, para una aplicación en tiempo real se debe garantizar un tiempo de cómputo: 

$$T_{computo}<N_{x}T_{s}$$

donde $N_{x}$ es el tamaño del bloque.

Las funciones para iniciar el controlador de DMA y la captura de un nuevo bloque dependen de la plataforma en particular donde se implemente la aplicación. En el ejemplo anterior, para ilustrar el concepto, se usó una rutina para capturar los datos por DMA con la siguiente sintaxis:

``captureBlock( void (*callback) () )``

En este prototipo ``callback`` es el puntero a la función que se invocará una vez finaliza la rutina la captura.

En el caso de la tarjeta ESP32, la cual soporta DMA para adquisición del ADC y escritura en el DAC, la implementación real del algoritmo anterior usando el sistema operativo de tiempo real freeRTOS luce como:

```C
// Implementación del filtro digital FIR usando DMA para una ESP32 y sistema operativo freeRTOS
// Este programa ha sido adaptado del ejemplo disponible en https://docs.espressif.com/projects/esp-idf/en/latest/esp32/api-reference/peripherals/adc_continuous.html

#include "freertos/FreeRTOS.h"
#include "freertos/task.h"
#include "esp_adc/adc_continuous.h"

//Tamaño máximo de la piscina de bloques que garantizan la captura continua
#define POOL_LEN                   1024

//Tamaño del bloque
#define READ_LEN                    256

static adc_channel_t channel[2] = {ADC_CHANNEL_6, ADC_CHANNEL_7};

static TaskHandle_t s_task_handle;

//ISR por fin de bloque. Genera un evento para la tarea de filtrado
static bool IRAM_ATTR s_conv_done_cb(adc_continuous_handle_t handle, const adc_continuous_evt_data_t *edata, void *user_data) {
    BaseType_t mustYield = pdFALSE;
    //Notify that ADC continuous driver has done enough number of conversions
    vTaskNotifyGiveFromISR(s_task_handle, &mustYield);
    return (mustYield == pdTRUE);
}

// Función auxiliar para configurar el controlador de DMA
static void continuous_adc_init(adc_channel_t *channel, uint8_t channel_num, adc_continuous_handle_t *out_handle) {
    adc_continuous_handle_t handle = NULL;

    adc_continuous_handle_cfg_t adc_config = {
        .max_store_buf_size = POOL_LEN,
        .conv_frame_size = READ_LEN,
    };
    ESP_ERROR_CHECK(adc_continuous_new_handle(&adc_config, &handle));

    adc_continuous_config_t dig_cfg = {
        .sample_freq_hz = 20 * 1000,
        .conv_mode = ADC_CONV_SINGLE_UNIT_1,
        .format = ADC_DIGI_OUTPUT_FORMAT_TYPE1,
    };

    adc_digi_pattern_config_t adc_pattern[SOC_ADC_PATT_LEN_MAX] = {0};
    dig_cfg.pattern_num = channel_num;
    for (int i = 0; i < channel_num; i++) {
        adc_pattern[i].atten = ADC_ATTEN_DB_0;
        adc_pattern[i].channel = channel[i] & 0x7;
        adc_pattern[i].unit = ADC_UNIT_1;
        adc_pattern[i].bit_width = SOC_ADC_DIGI_MAX_BITWIDTH;
    }
    dig_cfg.adc_pattern = adc_pattern;
    ESP_ERROR_CHECK(adc_continuous_config(handle, &dig_cfg));

    *out_handle = handle;
}

void setup() {
        //Configura el controlador de DMA & la ISR que se invoca cuando la captura termina
        adc_continuous_handle_t handle = NULL;
        continuous_adc_init(channel, sizeof(channel) / sizeof(adc_channel_t), &handle);

        adc_continuous_evt_cbs_t cbs = {
                .on_conv_done = s_conv_done_cb,
        };
        ESP_ERROR_CHECK(adc_continuous_register_event_callbacks(handle, &cbs, NULL));
        ESP_ERROR_CHECK(adc_continuous_start(handle));

        //Crea la tarea para realizar el filtrado
        xTaskCreate(
                TaskFilter, "Filter",
                2048, NULL, //Stack
                2, //High priority
                NULL
        );
}

short x_blk[NBLOQUE + Nh-1];
short y_blk[NBLOQUE];

void TaskFilter(void *pvParameters) {
        esp_err_t ret;
        uint32_t ret_num = 0;
        uint8_t result[READ_LEN] = {0};
        short x_new[READ_LEN];
        int n;
        
        while(1) {
                ulTaskNotifyTake(pdTRUE, portMAX_DELAY);
                ret = adc_continuous_read(handle, result, READ_LEN, &ret_num, 0);
                if (ret == ESP_OK) {
                        n = 0;
                        for (int i = 0; i < ret_num; i += SOC_ADC_DIGI_RESULT_BYTES) {
                                adc_digi_output_data_t *p = (adc_digi_output_data_t*)&result[i];
                                uint32_t chan_num = p->type1.channel;
                                uint32_t data = p->type1.data;
                                x_new[n++] = data;
                        }
                        dma_callbackfnc(x_new);
                }
        }
}

//Rutina que se llama cada vez que finaliza la captura de un bloque
void dma_callbackfnc(short* x_new) {
        int n, k;
        float y;

        //Copia el nuevo bloque de datos en x_blk a partir de la posición Nh-1 
        //con el fin de reservar las primeras posiciones para los elementos del bloque anterior
        //En ciertas plataformas, este proceso de copia se puede eliminar indicándole al 
        //hardware de DMA que inicie la captura a partir de la posición Nh-1 del bloque x_blk.
        for(n=0; n<NBLOQUE; n++) {
                x_blk[n+Nh-1] = x_new[n];         
        }

        //Calcula las Nh convoluciones. El ajuste del índice Nh-1 se usa para evitar el acceso 
        //a índices negativos del buffer de entrada que corresponden a las muestras del bloque pasado 
        for(n=0; n<NBLOQUE; n++)  {
                y = 0;
                for(k=0; k<Nh; k++)  {
                        y += h[k] * x_blk[n-k+Nh-1];
                }
                y_blk[n] = y;
        }
        //Copia los últimos elementos del bloque de entrada al principio para preparar
        //el bloque para la siguiente iteración
        for(k=0; k<Nh-1; k++)  {
                x_blk[k] = x_blk[k+NBLOQUE];
        }
}
```

Una forma alternativa de probar la implementación en DMA es usar un smartphone y la librería SimDSP for Android disponible en https://github.com/jimarinh/SimDSP-FIRtest